<a href="https://colab.research.google.com/github/nxxk23/NAKBID/blob/main/NINE/web_scrap_drug_center.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import shutil
import tempfile
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from neo4j import GraphDatabase

def login_and_get_download_url(username, password, login_url, download_page):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)

    try:
        driver.get(login_url)
        wait.until(EC.presence_of_element_located((By.NAME, "log"))).send_keys(username)
        wait.until(EC.presence_of_element_located((By.NAME, "pwd"))).send_keys(password)
        wait.until(EC.element_to_be_clickable((By.ID, "wp-submit"))).click()
        wait.until(EC.url_changes(login_url))
        print("✅ ล็อกอินสำเร็จ")

        driver.get(download_page)
        download_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.t-button.t-download")))
        download_url = download_button.get_attribute("href")

        if not download_url:
            raise ValueError("❌ ไม่พบลิงก์ดาวน์โหลด")

        cookies = {cookie['name']: cookie['value'] for cookie in driver.get_cookies()}
        return download_url, cookies
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดขณะล็อกอิน: {e}")
        return None, None
    finally:
        driver.quit()

def download_zip_to_temp(download_url, cookies):
    if not download_url:
        raise ValueError("❌ ไม่มี URL สำหรับดาวน์โหลด")

    headers = {"User-Agent": "Mozilla/5.0"}
    session = requests.Session()
    session.cookies.update(cookies)

    response = session.get(download_url, stream=True, headers=headers)
    if response.status_code == 200:
        temp_zip = tempfile.NamedTemporaryFile(delete=False, suffix=".zip")
        with open(temp_zip.name, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"✅ ดาวน์โหลดสำเร็จ: {temp_zip.name}")
        return temp_zip.name
    else:
        raise Exception(f"❌ ดาวน์โหลดล้มเหลว (Error {response.status_code})")

def extract_latest_files(zip_path):
    temp_dir = tempfile.mkdtemp()
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        print(f"✅ แตกไฟล์ ZIP ไปยัง {temp_dir}")
        return temp_dir
    except zipfile.BadZipFile:
        print("❌ ไฟล์ ZIP เสียหาย")
        return None

def load_excel_files(base_path, date_suffix):
    files = {
        "GP": f"{base_path}/Concept/GP{date_suffix}.xls",
        "VTM": f"{base_path}/Concept/VTM{date_suffix}.xls",
        "SUBS": f"{base_path}/Concept/SUBS{date_suffix}.xls",
        "TP": f"{base_path}/Concept/TP{date_suffix}.xls",
        "TPU": f"{base_path}/Concept/TPU{date_suffix}.xls",
        "GPU": f"{base_path}/Concept/GPU{date_suffix}.xls",
        "tp_tpu": f"{base_path}/Relationship/TPtoTPU{date_suffix}.xls",
        "gp_tp": f"{base_path}/Relationship/GPtoTP{date_suffix}.xls",
        "vtm_gp": f"{base_path}/Relationship/VTMtoGP{date_suffix}.xls",
        "subs_vtm": f"{base_path}/Relationship/SUBStoVTM{date_suffix}.xls",
        "gp_gpu": f"{base_path}/Relationship/GPtoGPU{date_suffix}.xls"
    }

    data = {}
    for key, path in files.items():
        if os.path.exists(path):
            try:
                data[key] = pd.read_excel(path)
                print(f"✅ โหลดไฟล์ {key} สำเร็จ")
            except Exception as e:
                print(f"❌ โหลดไฟล์ {key} ล้มเหลว: {e}")

    return data

def process_data(data):
    merge_df = data['tp_tpu'].merge(data['gp_tp'], on='TPID', how='left')
    merge_df = merge_df.merge(data['vtm_gp'], on='GPID', how='left')
    merge_df = merge_df.merge(data['subs_vtm'], on='VTMID', how='left')
    merge_df = merge_df.merge(data['gp_gpu'][['GPID', 'GPUID']], on='GPID', how='left')
    merge_df = merge_df.rename(columns={
        'GPID': 'TMTID(GP)', 'TPID': 'TMTID(TP)', 'GPUID': 'TMTID(GPU)', 'TPUID': 'TMTID(TPU)',
        'VTMID': 'TMTID(VTM)', 'SUBSID': 'TMTID(SUBS)'
    })
    for key in ['GP', 'TP', 'GPU', 'TPU', 'VTM', 'SUBS']:
        merge_df = merge_df.merge(data[key][['TMTID(' + key + ')', 'FSN']], on='TMTID(' + key + ')', how='left')
        merge_df = merge_df.rename(columns={'FSN': key + 'NAME'})
    merge_df = merge_df.groupby(['TMTID(TPU)', 'TPUNAME','TMTID(TP)', 'TPNAME', 'TMTID(GPU)', 'GPUNAME',
                                   'TMTID(GP)', 'GPNAME','TMTID(VTM)','VTMNAME'], as_index=False).agg({
        'TMTID(SUBS)': list, 'SUBSNAME': list
    })
    merge_df = merge_df.rename(columns={'TMTID(SUBS)': 'SUBSID_LIST', 'SUBSNAME': 'SUBS_LIST'})
    print(merge_df)
    return merge_df

def update_neo4j(df):
    uri = "neo4j+s://9bad2ad0.databases.neo4j.io"
    user = "neo4j"
    password = "pQdU1M3A-SHlORRRwi6ZOa1lYaEXhf_apicCIkGYfH0"
    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        session.run("""
            CREATE CONSTRAINT IF NOT EXISTS ON (n:DRUG) ASSERT n.`TMTID(GP)` IS UNIQUE;
            CREATE CONSTRAINT IF NOT EXISTS ON (n:SUBS) ASSERT n.`TMTID(SUBS)` IS UNIQUE;
            CREATE CONSTRAINT IF NOT EXISTS ON (n:VTM) ASSERT n.`TMTID(VTM)` IS UNIQUE;
            CREATE CONSTRAINT IF NOT EXISTS ON (n:TP) ASSERT n.`TMTID(TP)` IS UNIQUE;
            CREATE CONSTRAINT IF NOT EXISTS ON (n:GP) ASSERT n.`TMTID(GP)` IS UNIQUE;
        """)
        for _, row in df.iterrows():
            session.run("""
                MERGE (n:DRUG { `TMTID(GP)`: $gp_id })
                SET n += { `TMTID(TPU)`: $tpu_id, `TPUNAME`: $tpu_name, `TMTID(TP)`: $tp_id, `TPNAME`: $tp_name,
                           `TMTID(GPU)`: $gpu_id, `GPUNAME`: $gpu_name, `GPNAME`: $gp_name,
                           `TMTID(VTM)`: $vtm_id, `VTMNAME`: $vtm_name, `SUBSID_LIST`: $subs_list, `SUBS_LIST`: $subs_names }

                MERGE (s:SUBS { `TMTID(SUBS)`: $subs_list })
                SET s.`SUBSNAME` = $subs_names

                MERGE (v:VTM { `TMTID(VTM)`: $vtm_id })
                SET v.`VTMNAME` = $vtm_name

                MERGE (g:GP { `TMTID(GP)`: $gp_id })
                SET g.`GPNAME` = $gp_name

                MERGE (t:TP { `TMTID(TP)`: $tp_id })
                SET t.`TPNAME` = $tp_name

                MERGE (n)-[:HAS_SUBSTANCE]->(s)
                MERGE (n)-[:CONTAINS_VTM]->(v)
                MERGE (n)-[:HAS_GENERIC_NAME]->(g)
                MERGE (g)-[:HAS_TRADE_NAME]->(t)
            """,
            gp_id=row['TMTID(GP)'], tpu_id=row['TMTID(TPU)'], tpu_name=row['TPUNAME'],
            tp_id=row['TMTID(TP)'], tp_name=row['TPNAME'], gpu_id=row['TMTID(GPU)'],
            gpu_name=row['GPUNAME'], gp_name=row['GPNAME'], vtm_id=row['TMTID(VTM)'],
            vtm_name=row['VTMNAME'], subs_list=row['SUBSID_LIST'], subs_names=row['SUBS_LIST']
        )
    driver.close()


def main():
    username = "chanatip0615@gmail.com"
    password = "#chanatip2011"
    login_url = "https://www.this.or.th/en/account/"
    download_page = "https://www.this.or.th/en/download/"

    try:
        download_url, cookies = login_and_get_download_url(username, password, login_url, download_page)
        if not download_url:
            return

        temp_zip_path = download_zip_to_temp(download_url, cookies)
        temp_extract_dir = extract_latest_files(temp_zip_path)

        if not temp_extract_dir:
            return

        data = load_excel_files(temp_extract_dir, "20250303")
        print("this is a data :",data)
        if not data:
            return

        processed_data = process_data(data)
        update_neo4j(processed_data)

    except Exception as e:
        print(f"❌ พบข้อผิดพลาด: {e}")

    finally:
        if os.path.exists(temp_zip_path):
            os.remove(temp_zip_path)
            print("🗑️ ลบไฟล์ ZIP สำเร็จ")

        if os.path.exists(temp_extract_dir):
            shutil.rmtree(temp_extract_dir)
            print("🗑️ ลบโฟลเดอร์ชั่วคราวสำเร็จ")

if __name__ == "__main__":
    main()
